In [1]:
from TSForecasting.TsForecasting import TimeSeriesForecast

Using TensorFlow backend.


In [2]:
import dateparser

ModuleNotFoundError: No module named 'dateparser'

In [2]:
import dateutil.parser as dparser

In [22]:
dparser.parse("201612261047",fuzzy=True)

datetime.datetime(2016, 12, 26, 10, 47)

In [23]:
s = "2016:12:26:10:47"
k = ''.join(e for e in s if e.isalnum())
k

'201612261047'

In [25]:
z = dparser.parse(k,fuzzy=True)

In [28]:
type(z)

datetime.datetime

In [27]:
z.year

2016

In [92]:
import pandas as pd
import numpy as np
import dateutil.parser as dparser

In [97]:
cgmdata = pd.read_csv("~/Desktop/NCSA_genomics/Data/CGManalyzer.csv") #use your path

In [98]:
length = cgmdata.shape[0]
length

29189

In [99]:
for i in range(0,length):
    s = str(cgmdata.iloc[i]['timeStamp'])
    k=None
    k = ''.join(e for e in s if e.isalnum())
    z = dparser.parse(k,fuzzy=True)
    x = ""+str(z.year)+"-"+str(z.month)+"-"+str(z.day)+" "+str(z.hour)+":"+str(z.minute)+":"+str(z.second)
    cgmdata = cgmdata.replace(to_replace = s, value = x)

In [100]:
cgmdata

,ID,timeStamp,glucoseValue
0,ID01,2016-6-25 9:14:0,6.000000
1,ID01,2016-6-25 9:17:0,6.000000
2,ID01,2016-6-25 9:20:0,6.000000
3,ID01,2016-6-25 9:23:0,5.981000
4,ID01,2016-6-25 9:26:0,5.972000
...,...,...,...
29184,ID31,2016-6-9 16:49:0,4.161802
29185,ID31,2016-6-9 16:52:0,4.104440
29186,ID31,2016-6-9 16:55:0,4.013856
29187,ID31,2016-6-9 16:58:0,3.895511


In [64]:
cgmdata.iloc[0]['timeStamp']

'2016:06:25:09:14'

In [67]:
s = str(cgmdata.iloc[0]['timeStamp'])
k = ''.join(e for e in s if e.isalnum())
z = dparser.parse(k,fuzzy=True)
x = ""+str(z.year)+"-"+str(z.month)+"-"+str(z.day)+" "+str(z.hour)+":"+str(z.minute)+":"+str(z.second)
cgmdata = cgmdata.replace(to_replace = s, value = x)

In [7]:
from datetime import datetime

In [9]:
s = "20160625091456"
k = ''.join(e for e in s if e.isalnum())
z = dparser.parse(k,fuzzy=True)
x = ""+str(z.year)+"-"+str(z.month)+"-"+str(z.day)+" "+str(z.hour)+":"+str(z.minute)+":"+str(z.second)
y = datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
print(y)

2016-06-25 09:14:56


'2016-6-25 9:14:0'